In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [5]:
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [6]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [7]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL.

In [9]:
print(len(sentences))

795538


In [10]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [12]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-07-28 17:26:19,537 : INFO : 'pattern' package not found; tag filters are not available for English
2019-07-28 17:26:19,541 : INFO : collecting all words and their counts
2019-07-28 17:26:19,542 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-28 17:26:19,581 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-07-28 17:26:19,618 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-07-28 17:26:19,656 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2019-07-28 17:26:19,694 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2019-07-28 17:26:19,730 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types


Training model...


2019-07-28 17:26:19,768 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2019-07-28 17:26:19,805 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2019-07-28 17:26:19,847 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2019-07-28 17:26:19,888 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2019-07-28 17:26:19,928 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-07-28 17:26:19,969 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-07-28 17:26:20,011 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2019-07-28 17:26:20,052 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keeping 55847 word types
2019-07-28 17:26:20,092 : INFO : PROGRESS: at sentence #140000, processed 3107005 words, kee

2019-07-28 17:26:22,815 : INFO : PROGRESS: at sentence #780000, processed 17448092 words, keeping 122403 word types
2019-07-28 17:26:22,859 : INFO : PROGRESS: at sentence #790000, processed 17675168 words, keeping 123067 word types
2019-07-28 17:26:22,883 : INFO : collected 123505 word types from a corpus of 17798269 raw words and 795538 sentences
2019-07-28 17:26:22,884 : INFO : Loading a fresh vocabulary
2019-07-28 17:26:22,955 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123505, drops 107015)
2019-07-28 17:26:22,956 : INFO : effective_min_count=40 leaves 17239123 word corpus (96% of original 17798269, drops 559146)
2019-07-28 17:26:22,998 : INFO : deleting the raw counts dictionary of 123505 items
2019-07-28 17:26:23,001 : INFO : sample=0.001 downsamples 48 most-common words
2019-07-28 17:26:23,002 : INFO : downsampling leaves estimated 12749797 word corpus (74.0% of prior 17239123)
2019-07-28 17:26:23,042 : INFO : estimated required memory for 16490 w

2019-07-28 17:27:13,518 : INFO : EPOCH 5 - PROGRESS: at 85.34% examples, 1205949 words/s, in_qsize 7, out_qsize 0
2019-07-28 17:27:14,519 : INFO : EPOCH 5 - PROGRESS: at 94.15% examples, 1197512 words/s, in_qsize 7, out_qsize 0
2019-07-28 17:27:15,139 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-28 17:27:15,143 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-28 17:27:15,148 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-28 17:27:15,156 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-28 17:27:15,157 : INFO : EPOCH - 5 : training on 17798269 raw words (12749026 effective words) took 10.7s, 1196283 effective words/s
2019-07-28 17:27:15,157 : INFO : training on a 88991345 raw words (63750615 effective words) took 49.3s, 1292533 effective words/s
2019-07-28 17:27:15,158 : INFO : precomputing L2-norms of word weight vectors
2019-07-28 17:27:15,180 : INFO : saving Word2Vec object un

In [13]:
model.doesnt_match("man woman child kitchen".split())

/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [14]:
model.doesnt_match("france england germany berlin".split())

/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [15]:
model.doesnt_match("paris berlin london austria".split())

/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [16]:
model.most_similar("man")

/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6157339811325073),
 ('lady', 0.5747684240341187),
 ('lad', 0.560234546661377),
 ('men', 0.524324357509613),
 ('monk', 0.5216001868247986),
 ('millionaire', 0.5119434595108032),
 ('guy', 0.5102024674415588),
 ('businessman', 0.4984987676143646),
 ('poet', 0.49637120962142944),
 ('chap', 0.49499279260635376)]

In [18]:
model.most_similar("awful")

/Users/kshitij/Desktop/kaggle/kaggle-practice/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7747534513473511),
 ('horrible', 0.7304965257644653),
 ('atrocious', 0.726999044418335),
 ('abysmal', 0.7137743234634399),
 ('dreadful', 0.7017510533332825),
 ('horrendous', 0.6809265613555908),
 ('appalling', 0.6632950305938721),
 ('horrid', 0.6555044651031494),
 ('lousy', 0.6294952630996704),
 ('laughable', 0.6090008020401001)]